In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lang-8/lang8_to_m2.py
/kaggle/input/lang-8/readme.txt
/kaggle/input/lang-8/lang8.train.auto.bea19.m2


**IMPORTING LIBRARIES AND MODEL**

In [2]:
!pip install happytransformer
import re
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import STOPWORDS, WordCloud
from happytransformer import HappyTextToText, TTSettings, TTTrainArgs
from transformers import T5Tokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

**CONVERSION OF M2 FORMAT TO TXT**

In [3]:
def main():
    """
    this function stores the correct sentence line by line in as txt file with file name lang8.train.auto.bea19.m2
    """
    
    m2 = open("/kaggle/input/lang-8/lang8.train.auto.bea19.m2").read().strip().split("\n\n")
    out = open("fce.train.gold.bea19.txt", "w")
    # Do not apply edits with these error types
    skip = {"noop", "UNK", "Um"}
    
    for sent in m2:
        sent = sent.split("\n")
        cor_sent = sent[0].split()[1:] # Ignore "S "
        edits = sent[1:]
        offset = 0
        for edit in edits:
            edit = edit.split("|||")
            if edit[1] in skip: continue # Ignore certain edits
            coder = int(edit[-1])
            if coder != 0: continue # Ignore other coders
            span = edit[0].split()[1:] # Ignore "A "
            start = int(span[0])
            end = int(span[1])
            cor = edit[2].split()
            cor_sent[start+offset:end+offset] = cor
            offset = offset-(end-start)+len(cor)
        out.write(" ".join(cor_sent)+"\n")
        
main()


**PREPROCESSING INCORRECT SENTENCE**

In [4]:

file1 = open("/kaggle/input/lang-8/lang8.train.auto.bea19.m2","r")
s1 = file1.read()

each_sent = s1.split("\n\n")

incorrect = []
for i in range(len(each_sent)):
    temp = each_sent[i].split("\n")
    temp = temp[0]
    temp = temp.split(" ")
    temp = temp[1:]# ignore S
    temp = ' '.join(temp)
    incorrect.append(temp)
    

**PREOROCESSING CORRECT SENTENCE**

In [5]:
file2 = open("fce.train.gold.bea19.txt","r")
s2 = file2.read()

correct = s2.split("\n")


**STORING CORRECT AND INCORRECT SENTENCE IN DATAFRAME**

In [6]:
df = pd.DataFrame()
df["target"] = correct
df["input"] = incorrect


**STORE IN CSV FORMAT**

In [7]:
df.to_csv("data.csv",index=False)

index = []
for i in range(len(df.values)):
    if df.values[i][0] == df.values[i][1]:
        index.append(i)
            
df = df.drop(index)
df.shape



(498360, 2)

**CHECK NULL VALUES**

In [8]:
df.isnull().values.any()

df.dropna(inplace=True)

**CHECK DUPLICATE VALUE**

In [9]:
df.duplicated().values.any()

df.drop_duplicates(inplace=True)
df.reset_index(inplace=True,drop=True)

**TEXT CLEANING**

In [10]:
def clean(text):
   # removes characters inside (),{},[] and <>
   # removes characters like -+@#^/|*(){}$~`
   # we not not removing ,.!-:;"'

    text = re.sub('<.*>', '', text)
    text = re.sub('\(.*\)', '', text)
    text = re.sub('\[.*\]', '', text)
    text = re.sub('{.*}', '', text)
    text = re.sub("[-+@#^/|*(){}$~`<>=_]","",text)
    text = text.replace("\\","")
    text = re.sub("\[","",text)
    text = re.sub("\]","",text)
    text = re.sub("[0-9]","",text)
    return text



**LOWERCASE CONVERSION**

In [11]:
def word_cloud(df,col):

    comment_words = '' 
    stopwords = set(STOPWORDS) 

# iterate through the csv file 
    for val in df[col]:
        val = str(val) 
    # split the value 
        tokens = val.split() 
    # Converts each token into lowercase 
        for i in range(len(tokens)): 
            tokens[i] = tokens[i].lower() 

        comment_words += " ".join(tokens)+" "


**SPLIT DATA INTO TRAIN AND EVALUATION**

In [12]:
train_ratio = 0.8
train_size = int(train_ratio * len(df))
train_data = df[:train_size]
eval_data = df[train_size:]

train_data.to_csv("train.csv", index=False)
eval_data.to_csv("eval.csv", index=False)

**TRUNCATE AND LOAD TRAIN DATASET**

In [13]:
df = pd.read_csv("train.csv")

desired_length = 250000
truncated_df = df[:desired_length]

truncated_df.to_csv("t_train.csv", index=False)

**TOKENIZATION**

In [14]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

input_sentences = train_data["input"].tolist()
input_encodings = tokenizer(input_sentences, padding=True, truncation=True, return_tensors="pt")

target_sentences = train_data["target"].tolist()
target_encodings = tokenizer(target_sentences, padding=True, truncation=True, return_tensors="pt")

train_data["input_ids"] = input_encodings.input_ids.tolist() 
train_data["attention_mask"] = input_encodings.attention_mask.tolist() 
train_data["target_ids"] = target_encodings.input_ids.tolist()  
train_data["target_attention_mask"] = target_encodings.attention_mask.tolist() 


/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


**MODEL TRAINING**

In [15]:
happy_tt = HappyTextToText("T5", "t5-base")


args = TTTrainArgs(batch_size=8)
happy_tt.train("train.csv", args=args)

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-7e3b84767981e1a2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/398 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.781400
1000,0.692000
1500,0.669100
2000,0.672700
2500,0.641900
3000,0.642600
3500,0.631300
4000,0.629200
4500,0.612600
5000,0.616300


**BEAM SEARCH ALGORITHM**

In [16]:
def generate_text(input_text, happy_tt, tokenizer, beam_settings):
    input_text = "grammar: " + input_text
    result = happy_tt.generate_text(input_text, args=beam_settings)
    return result.text

beam_settings =  TTSettings(num_beams=2, min_length=1, max_length=50)

**EXAMPLES**

In [17]:
example_1 = "my email id is ramya@gmail.com"
result_1 = generate_text(example_1, happy_tt, tokenizer, beam_settings)
print(result_1)

grammar : my email id is ramya@gmail.com


In [18]:
example_2 = "I am enjoys, writtings articles ons AI and I also enjoyed write articling on AI."
result_2 = generate_text(example_2, happy_tt, tokenizer, beam_settings)
print(result_2)


grammar : I enjoy writing articles on AI and I also enjoy writing articles on AI .


In [19]:
import pickle

In [20]:
pickle.dump(happy_tt, open('/kaggle/working/model','wb'))

In [21]:
pip install joblib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [22]:
import joblib

# Assuming happy_tt is the object you want to save

# To save the object using joblib
joblib.dump(happy_tt, '/kaggle/working/model.joblib')

# To load the object from the saved file
loaded_happy_tt = joblib.load('/kaggle/working/model.joblib')
